In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Submodel(nn.Module):
    def __init__(self):
        super(Submodel, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5, 1)
        self.conv2 = nn.Conv2d(10, 20, 5, 1)
        self.dropout = nn.Dropout(0.5)
        self.lin1 = nn.Linear(320,50)
        self.lin2 = nn.Linear(50,10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = self.lin1(x)
        x = F.relu(x)
        return x

In [4]:
#load model with pretrained weights
submodel = Submodel()
submodel.eval()
PATH = 'models/base.pt'
submodel.load_state_dict(torch.load(PATH))

<All keys matched successfully>